In [ ]:
from Bio.PDB import PDBList                  # fetches/saves PDB data
from Bio.PDB.MMCIF2Dict import MMCIF2Dict    # parses data in mmCIF files
from Bio.PDB import MMCIFParser, Selection, is_aa
from Bio.SeqUtils import seq1, nt_search
import pandas as pd
import os
import numpy as  np

first part, load in mmcif file

In [ ]:
# load structure
structureDirectory = '/home/allen/projects/DATA/db/assemblies'
structureFile = '1p3p-assembly1.cif'
code = '1p3p'

parser = MMCIFParser(QUIET=True)
structure = parser.get_structure(code,os.path.join(structureDirectory,structureFile))

In [ ]:
# list number of models, and number of chains in model 0
print(len(structure),'model(s)')
chains = []
model = structure[0]
for chain in model:
    chains.append( chain.get_id() )
print(len(model),'chain(s) in model 0:',chains)

create the protein structure and sequence arrays

In [ ]:
# each entry in the dictionary will be a structure/sequence for a chain
proteinStructureDict = {}
proteinSequenceDict = {}
for chainid in chains:   # loop through all chains in model 0
    print('processing chain', chainid)
    xyzChain=[]  # will store the coords for the entire chain
    seq=[]       # will store sequence for the chain
    for residue in model[chainid]:
        if not is_aa(residue):   # skip if not aa residue
            continue
        xyzRes=np.ones((15,3))*np.nan    # blank residue array
        for i,atom in enumerate(residue):
            xyzRes[i]=atom.get_coord()
        xyzChain.append(xyzRes)
        seq.append(residue.get_resname())
    if xyzChain:   # only add to dict if an list is non-empty
        proteinStructureDict[chainid] = np.array(xyzChain)
        print('    added protein structure', end=', ')
    if seq:
        proteinSequenceDict[chainid] = seq1(''.join(seq))
        print('    added protein sequence')
print('completed processing all chains')
                

In [ ]:
proteinStructureDict.keys(), proteinSequenceDict.keys()

In [ ]:
proteinSequenceDict

In [ ]:
# create the oneHot array for the sequence
aminoAcids = "ARNDCEQGHILKMFPSTWYV"
oneHotDict = {}
for k, v in proteinSequenceDict.items():
    print('chain',k)
    print(v)
    pssm=np.zeros((len(v),20))
    pssm[ range(len(v)), [aminoAcids.find(c) for c in v]  ] =1
    oneHotDict[k] = pssm

In [ ]:
dnaSequence = {}
proteinChains
proteinSequence = { k:[] for k in chains }
for chainid in chains:
    print(chainid,'\n-----------------')
    aaList = [is_aa(res) for res in model[chainid]]
    
    if sum(aaList)>0: print('it\'s got protein!')
    print(aaList)
                

In [ ]:
for k, v in proteinStructureDict.items():
    print(k,v.shape)

In [ ]:
for k, v in proteinSequenceDict.items():
    print(k,len(v),v)

In [ ]:
for k, v in oneHotDict.items():
    print(k,v.shape)